In [1]:
import os
import sys
import glob
import cv2
import shutil
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

from preprocessing import *

from pathlib import Path

In [9]:
C1_df_path = "/home/aurora/tfg/data/upftfg26/apujols/processed/dataset_C1_M2024-2025.csv" 
C2_df_path = "/home/aurora/tfg/data/upftfg26/apujols/processed/dataset_C2_M2024-2025.csv"
dataset_path ="/home/aurora/tfg/data/upftfg26/apujols/processed/dataset.csv"

In [10]:
C1_df = pd.read_csv(C1_df_path, sep=";")
C2_df = pd.read_csv(C2_df_path, sep=";")

dataset = pd.concat([C1_df, C2_df], ignore_index=True)

dataset.to_csv(dataset_path, sep=";", index=False)

In [ ]:


folder_exists(output_folder)

if folder_exists(incoming_folder):
    # Get all the files
    video_files = glob.glob(f"{incoming_folder}/*.avi")
    xml_files = glob.glob(f"{incoming_folder}/*.xml")

    # Extract just the filenames
    video_names = [os.path.basename(f) for f in video_files]
    xml_names = [os.path.basename(f) for f in xml_files]

    print("VIDEOS:")
    for f in video_names:
        print(f)

    print("XMLS:")
    for f in xml_names:
        print(f)

✅ Folder exists: ../../../data/upftfg26/apujols/processed
✅ Folder exists: ../../../data/upftfg26/apujols/incoming
VIDEOS:
XMLS:


In [ ]:
sum_image = generate_sum_image(img_input_path=video_files[0], xml_input_path=xml_files[0], output_path=f"{output_folder}/sum_image")
sum_image_cropped, bbox, metadata = generate_cropped_sum_image(sum_img=sum_image,
                                                               img_input_path=video_files[0], 
                                                               xml_input_path=xml_files[0], 
                                                               output_path=f"{output_folder}/sum_image_cropped")
print_bounding_box(sum_image, bbox)